In [ ]:
from pyspark.sql.functions import col, to_timestamp, when

# Load Bronze
df_bronze = spark.read.format("delta").load("/delta/bronze/machine_failure")

# Clean and convert
df_silver = df_bronze \
    .withColumn("timestamp", to_timestamp("timestamp")) \
    .withColumn("is_failure", when(col("error_code").isNotNull(), 1).otherwise(0)) \
    .dropna(subset=["machine_id", "timestamp", "temperature"])

# Save as Silver
df_silver.write.format("delta").mode("overwrite").save("/delta/silver/machine_failure_clean")
